In [ ]:
import pandas as pd
from openai import OpenAI
import time
from tqdm.auto import tqdm
import signal
import sys
import os

class PromptProcessor:
    def __init__(self, df, api_key, output_path, model):
        self.df = df.copy()
        self.api_key = api_key
        self.output_path = output_path
        self.model = model
        self.client = OpenAI(api_key=api_key)
        self.is_interrupted = False
        
        if 'GPTResponse' not in self.df.columns:
            self.df['GPTResponse'] = ''
            
        signal.signal(signal.SIGINT, self.handle_interrupt)
        signal.signal(signal.SIGTERM, self.handle_interrupt)

    def handle_interrupt(self, signum, frame):
        print("\n\nInterruption received. Shutting down gracefully...")
        self.is_interrupted = True

    def save_progress(self):
        self.df.to_csv(self.output_path, index=False)

    def process_prompts(self):
        pbar = tqdm(total=len(self.df), desc="Processing prompts")
        
        try:
            for idx in self.df.index:
                if self.is_interrupted:
                    break
                    
                if pd.notna(self.df.loc[idx, 'GPTResponse']):
                    pbar.update(1)
                    continue
                
                try:
                    message = self.client.chat.completions.create(
                        model=self.model,
                        max_tokens=1024,
                        messages=[{
                            "role": "user",
                            "content": self.df.loc[idx, 'Prompt']
                        }]
                    )
                    
                    self.df.loc[idx, 'GPTResponse'] = message.choices[0].message.content
                    
                except Exception as e:
                    print(f"\nError processing prompt {idx}: {str(e)}")
                    self.df.loc[idx, 'GPTResponse'] = f"Error: {str(e)}"
                
                self.save_progress()
                pbar.update(1)
                
        finally:
            pbar.close()
            self.save_progress()
            
        return self.df

def process_prompts_file(input_path, output_path, api_key, model="gpt-4"):
    """
    Process prompts from input CSV and save results.
    
    Args:
        input_path (str): Path to input CSV file containing prompts
        output_path (str): Path to save results
        api_key (str): OpenAI API key
        model (str): GPT model to use (default: gpt-4)
    """
    if os.path.exists(output_path):
        print(f"Resuming from {output_path}")
        df = pd.read_csv(output_path)
    else:
        print(f"Starting new processing from {input_path}")
        df = pd.read_csv(input_path)
    
    processor = PromptProcessor(df, api_key, output_path, model)
    return processor.process_prompts()

In [ ]:
results_df = process_prompts_file(input_path="../results/results_v1.csv", output_path='../results/results_chatgpt.csv', api_key=api_key, model="gpt-4o-2024-11-20")